In [18]:
#!pip install numpy pandas matplotlib scikit-learn tensorflow keras

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

import os


In [9]:
caminho_pasta = 'Base_dados'
lista_arquivos = [arquivo for arquivo in os.listdir(caminho_pasta) if os.path.isfile(os.path.join(caminho_pasta, arquivo))]

for arq in lista_arquivos:
    df = pd.read_csv(f'Base_dados/{arq}')
    # Verificar se a coluna 'Date' existe e definir como índice
    # if 'Date' in df.columns:
    #     df = df.set_index('Date')
    print(df.head())


   Unnamed: 0.5  Unnamed: 0.4  Unnamed: 0.3  Unnamed: 0.2  Unnamed: 0.1  \
0             0             0             0             0             0   
1             1             1             1             1             1   
2             2             2             2             2             2   
3             3             3             3             3             3   
4             4             4             4             4             4   

   Unnamed: 0        Date       Open       High        Low  ...      MACD  \
0           0  2020-01-02  26.674999  26.985001  26.584999  ...  0.000000   
1           1  2020-01-03  26.549999  26.955000  26.424999  ... -0.003590   
2           2  2020-01-06  26.700001  26.700001  26.225000  ... -0.034680   
3           3  2020-01-07  26.500000  26.514999  26.145000  ... -0.074598   
4           4  2020-01-08  26.379999  26.495001  26.020000  ... -0.124168   

   MACD_Histogram  BBL_20_2.0  BBM_20_2.0  BBU_20_2.0  BBB_20_2.0  BBP_20_2.0  \
0    